Now we parse the content of the JSON file with the QASM circuits

In [1]:
using ITensors

include("simulation/mpo_simulation.jl")
using .MPOSimulation
include("utils/utils.jl")
using .Utils
include("simulation/mps_simulation.jl")
using .MPSSimulation



In [61]:
# Simulation using MPS
json_filename = "subcircuits.json"  
circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);

basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 10000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_psis, observables = mpo_sequence_apply_mps(circuits_data,basis_gates, cutoff, maxdim, method);

Completed applying MPO sequence for subcircuit B1.
Completed applying MPO sequence for subcircuit B2.
Completed applying MPO sequence for subcircuit B3.
Completed applying MPO sequence for subcircuit B4.
Completed applying MPO sequence for subcircuit B5.
Completed applying MPO sequence for subcircuit B6.
Completed applying MPO sequence for subcircuit B7.
Completed applying MPO sequence for subcircuit B8.
Completed applying MPO sequence for subcircuit B9.
Completed applying MPO sequence for subcircuit B10.
Completed applying MPO sequence for subcircuit B11.
Completed applying MPO sequence for subcircuit B12.
Completed applying MPO sequence for subcircuit B13.
Completed applying MPO sequence for subcircuit B14.
Completed applying MPO sequence for subcircuit B15.
Completed applying MPO sequence for subcircuit B16.
Completed applying MPO sequence for subcircuit B17.
Completed applying MPO sequence for subcircuit B18.
Completed applying MPO sequence for subcircuit B19.
Completed applying MP

In [53]:
expval_tensors = 0.0
observable_keys = collect(keys(observables))
first_key = observable_keys[1]
num_observables = length(observables[first_key])

for (i, coeff) in enumerate(coefficients)
    expval_tensors += coeff * sum(prod(sum(prob * inner(psi, apply(observables[key][j], psi)) for (psi, prob) in all_psis[key][i]) for key in observable_keys) for j in 1:num_observables)
end

expval_tensors = real(expval_tensors)
discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits=4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits=8))
• Expected Value Tensors:  $(round(expval_tensors, digits=8))
• Discrepancy            : $discrepancy%
==============================
""")


• Expected Value Knitting: 19.18054219
• Expected Value Tensors:  19.18025228
• Discrepancy            : 0.0015%



In [60]:
json_filename = "subcircuits.json"  
circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);

basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 10000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_rhos, observables = mpo_sequence_apply(circuits_data,basis_gates, cutoff, maxdim, method);

Completed applying MPO sequence for subcircuit B1.
Completed applying MPO sequence for subcircuit B2.
Completed applying MPO sequence for subcircuit B3.
Completed applying MPO sequence for subcircuit B4.
Completed applying MPO sequence for subcircuit B5.
Completed applying MPO sequence for subcircuit B6.
Completed applying MPO sequence for subcircuit A1.
Completed applying MPO sequence for subcircuit A2.
Completed applying MPO sequence for subcircuit A3.
Completed applying MPO sequence for subcircuit A4.
Completed applying MPO sequence for subcircuit A5.
Completed applying MPO sequence for subcircuit A6.


In [57]:
expval_tensors = real(sum(coefficients[i] * (sum((inner(observables["A"][j],all_rhos["A"][i]) * inner(observables["B"][j],all_rhos["B"][i])) for j in 1:length(observables["A"]))) for i in 1:length(coefficients)))

discrepancy = abs(round(100*(expval_knitting - expval_tensors)/expval_knitting, digits = 4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits = 8))
• Expected Value Tensors:  $(round(expval_tensors, digits = 8))
• Discrepancy    : $discrepancy%
==============================
""")


• Expected Value Knitting: 23.02423374
• Expected Value Tensors:  23.02423349
• Discrepancy    : 0.0%



In [ ]:
# Time of simulation with different bound dimensions

basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
cutoff = 1e-12
maxdims = 10:4:30

for maxdim in maxdims
    elapsed_time = @elapsed begin
        all_mpo_sequences, all_rhos, observables = redirect_stdout(devnull) do
            mpo_sequence_apply(circuits_data, basis_gates, cutoff, maxdim, method)
        end

        expval_tensors = real(sum(coefficients[i] * sum(inner(observables["A"][j], all_rhos["A"][i]) * inner(observables["B"][j], all_rhos["B"][i]) for j in 1:length(observables["A"])) for i in 1:length(coefficients)))
        discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits = 4))
    end

    println("
    ==============================
    • Expected Value Knitting: $(round(expval_knitting, digits = 8))
    • Expected Value Tensors:  $(round(expval_tensors, digits = 8))
    • Discrepancy    : $discrepancy%
    • Max bound dimension  : $maxdim
    • Elapsed Time : $(round(elapsed_time, digits=2)) seconds
    ==============================
    ")
end



    • Expected Value Knitting: 15.33019463
    • Expected Value Tensors:  15.03390918
    • Discrepancy    : 1.9327%
    • Max bound dimension  : 10
    • Elapsed Time : 2.95 seconds
    

    • Expected Value Knitting: 15.33019463
    • Expected Value Tensors:  15.32272587
    • Discrepancy    : 0.0487%
    • Max bound dimension  : 14
    • Elapsed Time : 4.63 seconds
    

    • Expected Value Knitting: 15.33019463
    • Expected Value Tensors:  15.33019463
    • Discrepancy    : 0.0%
    • Max bound dimension  : 18
    • Elapsed Time : 3.27 seconds
    

    • Expected Value Knitting: 15.33019463
    • Expected Value Tensors:  15.33019463
    • Discrepancy    : 0.0%
    • Max bound dimension  : 22
    • Elapsed Time : 3.45 seconds
    

    • Expected Value Knitting: 15.33019463
    • Expected Value Tensors:  15.33019463
    • Discrepancy    : 0.0%
    • Max bound dimension  : 26
    • Elapsed Time : 3.05 seconds
    

    • Expected Value Knitting: 15.33019463
    • Expected Value